In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_type_code = df['Type_Code'].tolist()
list_part_key = df['Part_Key'].tolist()

print('总数量：' + str(len(list_part_key)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for type_code, part_key in zip(list_type_code, list_part_key):
    work.put_nowait((type_code, part_key))

def crawler():
    global df
    
    while not work.empty():
        type_code, part_key = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                if part_key == '':
                        df_temp = pd.DataFrame([{'status': 'ok',
                                                 'Part_Key': part_key,
                                                 'Vehicle': ''}])

                        status = 'ok'

                        break
                
                data = 'func=getbuyersguide&payload=' + quote('{"partData":{"listing_data_essential":{"parttype":"') + type_code + quote('","partkey":"') + part_key + quote('"}}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = html.xpath('//div[contains(@class, "buyersguide-nested")]/descendant::table[contains(@class, "nobmp")]/tr')

                vehicle = '\n'.join([' '.join(_.xpath('./td/text()')).strip() for _ in list_vehicle])

                # = = = = = = = = = = = = = = =
           
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Key': part_key,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Key': part_key}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_key +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：3

14774481  <->  [ok] - 剩余数量：0 - 当前时间：17:44:24
14773981  <->  [ok] - 剩余数量：0 - 当前时间：17:44:24
14774441  <->  [ok] - 剩余数量：0 - 当前时间：17:44:29

爬虫输出中

爬虫结束
